# plotting fit results

This notebook runs the function that produces publication ready fit plots

In [1]:
## imports and configuration
%cd '/home/naodell/work/wbr/analysis'
#%load_ext autoreload

from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.optimize import minimize
from tqdm.notebook import tqdm

import scripts.plot_tools as pt
import scripts.fit_helpers as fh

np.set_printoptions(precision=3)
rc_params = {
             'figure.figsize': (10, 10),
             'axes.labelsize': 20,
             'axes.facecolor': 'white',
             'axes.titlesize':'x-large',
             'legend.fontsize': 20,
             'xtick.labelsize':20,
             'ytick.labelsize':20,
             'font.size':18,
             'font.sans-serif':['Arial', 'sans-serif'],
             'mathtext.sf':'Arial',
             'lines.markersize':8.,
             'lines.linewidth':2.5,
            }
matplotlib.rcParams.update(rc_params)

%connect_info

/home/naodell/work/wbr/analysis
{
  "shell_port": 35451,
  "iopub_port": 37503,
  "stdin_port": 42173,
  "control_port": 48831,
  "hb_port": 44793,
  "ip": "127.0.0.1",
  "key": "16aa5c54-620ad998568cd1740e389128",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-5fe819d9-87f0-4a12-8e42-bf57ab79a44b.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
# configure, get the input data, and do any additional processing that is needed
processes = ['ttbar', 't', 'ww', 'wjets', 'zjets_alt', 'diboson', 'fakes'] 
selections = [
              'ee', 'mumu',  
              'emu', 
              'mutau', 'etau', 
              'mujet', 'ejet'
             ]
plot_labels   = fh.fancy_labels
bg_labels =  ['Z', 'VV (non-WW)', 'QCD']

# initialize fit data
infile = open('local_data/fit_data_asimov.pkl', 'rb')
fit_data = pickle.load(infile)
decay_map = fit_data._decay_map
ww_labels = decay_map.alt_label.values 

# get parameter data
parameters = fit_data._parameters

In [3]:
# make some control plots 
params_pre = parameters['val_init'].values
params_post = parameters['val_fit'].values
yields = dict()
binning_dict = dict()
for category, model_data in tqdm(fit_data._model_data.items()):
    # parse lepton selection and jet category
    cat_split = category.split('_')
    sel = cat_split[0]
    jet_cat = '_'.join(cat_split[1:])
     
    # build prefit expectation from model_tensor
    expected_pre, expected_var = fit_data.mixture_model(params_pre, category)
    
    # build postfit expectation from model_tensor
    expected_post, _ = fit_data._bb_np[category]*fit_data.mixture_model(params_post, category)

    # get the data and binning
    template_data = fit_data.get_selection_data(sel)[jet_cat]
    data_val, data_var = template_data['templates']['data']['val'], template_data['templates']['data']['var']
    bins = template_data['bins'][:-1]
    binning_dict[category] = bins
    
    # get template components (so ugly)
    expected_test, _ = fit_data.mixture_model(params_post, category, no_sum=True) # for testing template removal
    pmask = model_data['process_mask']
    labels = []
    signal_mixture = {dm_row['alt_label'] : np.zeros(expected_test.shape[1]) for irow, dm_row in decay_map.iterrows() if dm_row[sel] == 1}
    signal_mixture['other'] = np.zeros(expected_test.shape[1])
    #signal_mixture[r'$t\bar{t}/tW/WW\rightarrow other$'] = np.zeros(expected_test.shape[1])
    count = 0
    for i, pstate in enumerate(pmask):
        if pstate:
            if i < 63:
                label = ww_labels[i%21]
                if decay_map.loc[i%21 + 1, sel] == 1:
                    signal_mixture[label] += expected_test[count]
                else:
                    signal_mixture['other'] += expected_test[count]
                    #signal_mixture[r'$t\bar{t}/tW/WW\rightarrow other$'] += expected_test[count]
                count += 1
            if 63 <= i < 69:
                signal_mixture['W'] += expected_test[count]
                count += 1
            if i == 69:
                signal_mixture['Z'] = expected_test[count]
                labels.append('Z')
                count += 1
            if i == 70:
                signal_mixture['VV (non-WW)'] = expected_test[count]
                labels.append('VV (non-WW)')
                count += 1
            if i == 71:
                signal_mixture['QCD'] = expected_test[count]
                labels.append('QCD')
                count += 1
                
        if i == 62: # sort signal templates by size
            labels.extend([l[0] for l in sorted(signal_mixture.items(), key=lambda x: x[1].sum(), reverse=True)])
            signal_mixture['W'] = np.zeros(expected_test.shape[1])
            labels.append('W')
    
    # test systematic uncertainties
    ntoys = 10000
    pval, perr = params_pre[4:], parameters['err_init'].values[4:]
    prand = pval + np.random.randn(pval.size, ntoys).T*perr 
    toy_data = np.array([fit_data.mixture_model(np.concatenate([fit_data._beta_init, p]), category)[0] for p in prand]) 
    #toy_data = np.array([td*expected_pre.sum()/td.sum() for td in toy_data])
    toy_data.sort(axis=0)
    median = toy_data[ntoys//2]
    syst_err_pre = (toy_data[int(0.16*ntoys),:]-median, toy_data[-int(0.16*ntoys),:]-median)
    
    pval, perr = params_post[4:], parameters['err_fit'].values[4:]
    prand = pval + np.random.randn(pval.size, ntoys).T*perr 
    toy_data = np.array([fit_data.mixture_model(np.concatenate([fit_data._beta_init, p]), category)[0] for p in prand]) 
    toy_data.sort(axis=0)
    median = toy_data[ntoys//2]
    syst_err_post = (toy_data[int(0.16*ntoys),:]-median, toy_data[-int(0.16*ntoys),:]-median)
    
    #print(category)
    #print(expected_pre)
    #print(syst_err_pre[0])
    #print(syst_err_pre[1])
    #print(syst_err_post[0])
    #print(syst_err_post[1])

    pt.fit_plot(bins, model_data['data'][0], expected_pre, expected_post, 
                signal_mixture, labels,
                np.sqrt(expected_var), syst_err_pre, syst_err_post,
                xlabel=(plot_labels[sel][0]),
                title=f'{plot_labels[sel][1]}: {pt.categories[jet_cat].label}',
                output_path=f'plots/fits/{sel}_{jet_cat}.png',
                show=False
               )

f = open('local_data/binning.pkl', 'wb')
pickle.dump(binning_dict, f)
f.close()

findfont: Font family ['cursive'] not found. Falling back to DejaVu Sans.
